In [1]:
import pandas as pd

# 미국의 state별 면적 당 인구수를 구해보자

## 1. 데이터 불러오기

### 1.1 population data
- 지역, 연령대(total / under18), 데이터수집년도, 인구수를 보여주는 데이터

In [2]:
population = pd.read_csv("Data/state-population.csv", encoding = 'utf-8-sig')
population.head() 

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


### 1.2 areas data
- 각 state 별로 면적을 보여주는 데이터 

In [3]:
areas = pd.read_csv("Data/state-areas.csv", encoding = 'utf-8-sig')
areas.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


### 1.3 abbrevs data
- 각 state의 약어를 보여주는 데이터

In [4]:
abbrevs = pd.read_csv("Data/state-abbrevs.csv", encoding = 'utf-8-sig')
abbrevs.head()

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


## 2. 데이터 전처리

### 2.1.population data

In [5]:
pop = pd.read_csv("Data/state-population.csv", encoding = 'utf-8-sig')
pop.head()

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


#### 2.1.1. null 값 처리

In [6]:
pop.isnull().sum()

state/region     0
ages             0
year             0
population      20
dtype: int64

population column에 NaN값이 있음을 알 수 있다.  
그래서 population 열에서 NaN 값을 가지는 행을 가져와보면 다음과 같다. 

In [7]:
pop[pop['population'].isnull()]

,state/region,ages,year,population
2448,PR,under18,1990,NaN
2449,PR,total,1990,NaN
2450,PR,total,1991,NaN
2451,PR,under18,1991,NaN
2452,PR,total,1993,NaN
2453,PR,under18,1993,NaN
2454,PR,under18,1992,NaN
2455,PR,total,1992,NaN
2456,PR,under18,1994,NaN
2457,PR,total,1994,NaN


state/region 이 PR인 행에서 모든 NaN 값이 발생함을 알 수 있다.
그러면 이 NaN 값을 처리해줘야 하는데, 그 전에 PR의 정보를 알기 위해서 state/region 이 PR인 행에서 NaN이 아닌 데이터를 확인해보자.

In [8]:
PR = pop[pop['state/region']=='PR']
PR = PR[PR['population'].notnull()]

In [9]:
PR.head()

,state/region,ages,year,population
2468,PR,total,2000,3810605.0
2469,PR,under18,2000,1089063.0
2470,PR,total,2001,3818774.0
2471,PR,under18,2001,1077566.0
2472,PR,total,2002,3823701.0


In [10]:
PR.tail()

,state/region,ages,year,population
2491,PR,under18,2010,896945.0
2492,PR,under18,2011,869327.0
2493,PR,total,2011,3686580.0
2494,PR,under18,2012,841740.0
2495,PR,total,2012,3651545.0


이를 보면 PR 지역은 2000년부터 집계가 시작되었음을 알 수 있다.  
또한 위의 NaN데이터를 보면 1990~1999 까지는 NaN 값이고, 2000년부터는 중간에 끊긴 부분이 없기 때문에 PR에서는 그냥 NaN 값을 drop시키면 될것같다.

In [11]:
pop = pop.dropna()

In [12]:
pop.isnull().sum()

state/region    0
ages            0
year            0
population      0
dtype: int64

dropna 결과를 확인해보니 이제 NaN 값이 없는 것을 확인할 수 있다.
이제는 뒤에 있을 전처리 과정을 위해 열의 이름을 바꿔주자.  
abbrevs data 의 abbreviation 열에 지역의 약어가 들어가 있기 떄문에 population 데이터의 state/region 열을 맞춰주면 될것 같다.
population 데이터의 열을 리스트로 받아오고, 그 리스트에 접근하는 방법으로 열의 이름을 바꿔주었다. 

In [13]:
cols = pop.columns.to_list()
cols[0] = 'abbreviation'
pop.columns = cols
pop.columns

Index(['abbreviation', 'ages', 'year', 'population'], dtype='object')

NaN 값을 없애줬으니, 이제는 중복값이 있는지도 확인해보자.  
duplicated() 와 value_counts를 이용해보면 될것같다.

In [14]:
pop.duplicated().sum()

0

In [15]:
pop.value_counts().max()

1

중복값이 없다는 결과를 확인할 수 있다.

#### 2.1.2.  total / under18 분류하기
- 연령대별로 지표를 알아보기 위해 분류

In [16]:
pop_tot = pop[pop['ages'] == 'total']
pop_tot.head()

,abbreviation,ages,year,population
1,AL,total,2012,4817528.0
3,AL,total,2010,4785570.0
5,AL,total,2011,4801627.0
6,AL,total,2009,4757938.0
9,AL,total,2013,4833722.0


In [17]:
pop_18 = pop[pop['ages'] == 'under18']
pop_18.head()

,abbreviation,ages,year,population
0,AL,under18,2012,1117489.0
2,AL,under18,2010,1130966.0
4,AL,under18,2011,1125763.0
7,AL,under18,2009,1134192.0
8,AL,under18,2013,1111481.0


### 2.2.areas Data

In [18]:
area = pd.read_csv("Data/state-areas.csv", encoding = 'utf-8-sig')
area.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


In [19]:
area.shape

(52, 2)

#### 2.2.1. null값/ 중복값 처리하기

In [20]:
area.isnull().sum()

state            0
area (sq. mi)    0
dtype: int64

In [21]:
area.duplicated().sum()

0

NaN 값과 중복값 모두 없음을 확인할 수 있다. 기분이 좋다.

### 2.3. abbreves data

In [22]:
abb = pd.read_csv("Data/state-abbrevs.csv", encoding = 'utf-8-sig')
abb.head()

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [23]:
abb.shape

(51, 2)

#### 2.3.1. null값/ 중복값 처리하기

In [24]:
abb.isnull().sum()

state           0
abbreviation    0
dtype: int64

In [25]:
abb.duplicated().sum()

0

abbreves 데이터도 NaN 값과 중복값이 없는 깔끔한 데이터이다. 

## 3. area / abb 병합
- area 데이터의 state를 약어로 나타내기 위해 둘을 병합하여 정리해준다.

In [26]:
area.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


In [29]:
abb.head()

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


두 데이터에 공통으로 들어가있는 state 열을 기준으로 병합해주고, 모든 데이터를 가져오기 위해 outer join 해준다.  
병합 후에 두 데이터 중 하나의 데이터에만 있는 정보는 NaN 값이 될 가능성이 있기 떄문에 null 값 유무도 확인해야한다.

In [30]:
area_abb = pd.merge(area, abb, on = 'state', how = 'outer')
area_abb.head()

,state,area (sq. mi),abbreviation
0,Alabama,52423,AL
1,Alaska,656425,AK
2,Arizona,114006,AZ
3,Arkansas,53182,AR
4,California,163707,CA


In [31]:
area_abb.isnull().sum()

state            0
area (sq. mi)    0
abbreviation     1
dtype: int64

abbreviation 열에 NaN 값이 있다.  
어디서 나온 건지 확인해보자.

In [32]:
area_abb[area_abb['abbreviation'].isnull()]

,state,area (sq. mi),abbreviation
51,Puerto Rico,3515,NaN


확인 결과 Puetro Rico 의 약어가 정해지지 않았다.  
fillna() 함수를 통해 NaN 값을 PR로 채워주자.

In [33]:
area_abb = area_abb.fillna("PR")
area_abb[area_abb['state'] == 'Puerto Rico']

,state,area (sq. mi),abbreviation
51,Puerto Rico,3515,PR


## 4.population과 병합

### 4.1. pop_tot 과 병합

pop_tot 데이터와 df1 데이터에 공통으로 들어가있는 abbreviation 을 기준으로 병합하고, outer join 해주자.  
이 경우도 NaN 값이 있는지 확인해야한다.

In [34]:
final_tot = pd.merge(pop_tot, area_abb, on = 'abbreviation', how = 'outer')
final_tot

,abbreviation,ages,year,population,state,area (sq. mi)
0,AL,total,2012,4817528.0,Alabama,52423.0
1,AL,total,2010,4785570.0,Alabama,52423.0
2,AL,total,2011,4801627.0,Alabama,52423.0
3,AL,total,2009,4757938.0,Alabama,52423.0
4,AL,total,2013,4833722.0,Alabama,52423.0
...,...,...,...,...,...,...
1257,USA,total,2013,316128839.0,NaN,NaN
1258,USA,total,2009,306771529.0,NaN,NaN
1259,USA,total,2010,309326295.0,NaN,NaN
1260,USA,total,2011,311582564.0,NaN,NaN


In [35]:
final_tot.isnull().sum()

abbreviation      0
ages              0
year              0
population        0
state            24
area (sq. mi)    24
dtype: int64

NaN 값이 있기 때문에 어디서 나온 값인지 확인하자.

In [36]:
final_tot[final_tot['state'].isnull()]

,abbreviation,ages,year,population,state,area (sq. mi)
1238,USA,total,1990,249622814.0,NaN,NaN
1239,USA,total,1991,252980942.0,NaN,NaN
1240,USA,total,1992,256514231.0,NaN,NaN
1241,USA,total,1993,259918595.0,NaN,NaN
1242,USA,total,1994,263125826.0,NaN,NaN
1243,USA,total,1995,266278403.0,NaN,NaN
1244,USA,total,1996,269394291.0,NaN,NaN
1245,USA,total,1997,272646932.0,NaN,NaN
1246,USA,total,1998,275854116.0,NaN,NaN
1247,USA,total,2000,282162411.0,NaN,NaN


확인 결과 USA에서 NaN 값이 나온 것을 확인할 수 있다.  
하지만 우리가 구하려는 state 별 면적대비 인구수와는 관계가 없는 내용이므로 그냥 dropna()를 해서 없애주도록 하자.

In [37]:
final_tot = final_tot.dropna()
final_tot

,abbreviation,ages,year,population,state,area (sq. mi)
0,AL,total,2012,4817528.0,Alabama,52423.0
1,AL,total,2010,4785570.0,Alabama,52423.0
2,AL,total,2011,4801627.0,Alabama,52423.0
3,AL,total,2009,4757938.0,Alabama,52423.0
4,AL,total,2013,4833722.0,Alabama,52423.0
...,...,...,...,...,...,...
1233,PR,total,2013,3615086.0,Puerto Rico,3515.0
1234,PR,total,2009,3740410.0,Puerto Rico,3515.0
1235,PR,total,2010,3721208.0,Puerto Rico,3515.0
1236,PR,total,2011,3686580.0,Puerto Rico,3515.0


In [38]:
final_tot.isnull().sum()

abbreviation     0
ages             0
year             0
population       0
state            0
area (sq. mi)    0
dtype: int64

In [39]:
final_tot.duplicated().sum()

0

확인 결과 NaN 값과 중복값이 없는 깔끔한 데이터가 되었다.

### 4.2. pop_18 과 병합

이 경우 역시 USA 값이 NaN 이 나올 것이다.  
dropna() 를 해서 정리해주자.

In [40]:
final_18 = pd.merge(pop_18, area_abb, on = 'abbreviation', how = 'outer')
final_18

,abbreviation,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,under18,2010,1130966.0,Alabama,52423.0
2,AL,under18,2011,1125763.0,Alabama,52423.0
3,AL,under18,2009,1134192.0,Alabama,52423.0
4,AL,under18,2013,1111481.0,Alabama,52423.0
...,...,...,...,...,...,...
1257,USA,under18,2013,73585872.0,NaN,NaN
1258,USA,under18,2009,74134167.0,NaN,NaN
1259,USA,under18,2010,74119556.0,NaN,NaN
1260,USA,under18,2011,73902222.0,NaN,NaN


In [41]:
final_18.isnull().sum()

abbreviation      0
ages              0
year              0
population        0
state            24
area (sq. mi)    24
dtype: int64

In [42]:
final_18 = final_18.dropna()
final_18

,abbreviation,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,under18,2010,1130966.0,Alabama,52423.0
2,AL,under18,2011,1125763.0,Alabama,52423.0
3,AL,under18,2009,1134192.0,Alabama,52423.0
4,AL,under18,2013,1111481.0,Alabama,52423.0
...,...,...,...,...,...,...
1233,PR,under18,2013,814068.0,Puerto Rico,3515.0
1234,PR,under18,2009,920794.0,Puerto Rico,3515.0
1235,PR,under18,2010,896945.0,Puerto Rico,3515.0
1236,PR,under18,2011,869327.0,Puerto Rico,3515.0


## 5. 면적 당 인구수
- pop/area 라는 새로운 열을 만들어서 해당 내용을 넣어준다.

In [43]:
final_tot['pop/area'] = final_tot['population']/final_tot['area (sq. mi)']
final_tot

<ipython-input-43-817af97f792e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_tot['pop/area'] = final_tot['population']/final_tot['area (sq. mi)']


,abbreviation,ages,year,population,state,area (sq. mi),pop/area
0,AL,total,2012,4817528.0,Alabama,52423.0,91.897221
1,AL,total,2010,4785570.0,Alabama,52423.0,91.287603
2,AL,total,2011,4801627.0,Alabama,52423.0,91.593900
3,AL,total,2009,4757938.0,Alabama,52423.0,90.760506
4,AL,total,2013,4833722.0,Alabama,52423.0,92.206131
...,...,...,...,...,...,...,...
1233,PR,total,2013,3615086.0,Puerto Rico,3515.0,1028.473969
1234,PR,total,2009,3740410.0,Puerto Rico,3515.0,1064.128023
1235,PR,total,2010,3721208.0,Puerto Rico,3515.0,1058.665149
1236,PR,total,2011,3686580.0,Puerto Rico,3515.0,1048.813656


In [60]:
final_18['pop/area'] = final_18['population']/final_18['area (sq. mi)']
final_18

<ipython-input-60-b13c84d395ad>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_18['pop/area'] = final_18['population']/final_18['area (sq. mi)']


,abbreviation,ages,year,population,state,area (sq. mi),pop/area
0,AL,under18,2012,1117489.0,Alabama,52423.0,21.316769
1,AL,under18,2010,1130966.0,Alabama,52423.0,21.573851
2,AL,under18,2011,1125763.0,Alabama,52423.0,21.474601
3,AL,under18,2009,1134192.0,Alabama,52423.0,21.635389
4,AL,under18,2013,1111481.0,Alabama,52423.0,21.202163
...,...,...,...,...,...,...,...
1233,PR,under18,2013,814068.0,Puerto Rico,3515.0,231.598293
1234,PR,under18,2009,920794.0,Puerto Rico,3515.0,261.961309
1235,PR,under18,2010,896945.0,Puerto Rico,3515.0,255.176387
1236,PR,under18,2011,869327.0,Puerto Rico,3515.0,247.319203


모든 전처리가 끝났다!!  
이제부터는 필요한 부분만 가져와서 시각화를 하거나 분석하면 된다.

## 6. 2013년 / total에서 면적 당 인구수
- 면적당 인구수가 많은 순서대로 정렬해주자.

In [45]:
density_2013_tot = final_tot[final_tot['year']== 2013]
density_2013_tot = density_2013_tot.sort_values('pop/area', ascending=False)
density_2013_tot

,abbreviation,ages,year,population,state,area (sq. mi),pop/area
195,DC,total,2013,646449.0,District of Columbia,68.0,9506.602941
1233,PR,total,2013,3615086.0,Puerto Rico,3515.0,1028.473969
723,NJ,total,2013,8899339.0,New Jersey,8722.0,1020.332378
955,RI,total,2013,1051511.0,Rhode Island,1545.0,680.589644
147,CT,total,2013,3596080.0,Connecticut,5544.0,648.643579
524,MA,total,2013,6692824.0,Massachusetts,10555.0,634.090384
483,MD,total,2013,5928814.0,Maryland,12407.0,477.860401
187,DE,total,2013,925749.0,Delaware,1954.0,473.771238
772,NY,total,2013,19651127.0,New York,54475.0,360.736613
236,FL,total,2013,19552860.0,Florida,65758.0,297.345722


In [46]:
density_2013_tot_top10 = density_2013_tot[['abbreviation', 'pop/area']][:10]
density_2013_tot_top10

,abbreviation,pop/area
195,DC,9506.602941
1233,PR,1028.473969
723,NJ,1020.332378
955,RI,680.589644
147,CT,648.643579
524,MA,634.090384
483,MD,477.860401
187,DE,473.771238
772,NY,360.736613
236,FL,297.345722
